In [ ]:
!pip install xgboost
!pip install hyperopt
!pip install graphviz

In [ ]:
from sparc_to_galfit_feedme_gen import *
from xgboost_feedme_functions import galfit_param_grab, flatten_to_pandas, make_hist_plots, export_filter #build_df, choose_df, 

In [ ]:
import xgboost as xgb
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import os
from os.path import join as pj

In [ ]:
model_xgb = xgb.XGBRegressor()
model_xgb.load_model("xgboost_model.json")

In [ ]:
cwd = os.getcwd()
home_dir = os.environ['HOME']

In [ ]:
top_dir = pj(home_dir, "run5_galfit_xgboosted")
in_dir, tmp_dir, out_dir = command_line(top_dir)
filenames_fits_in, galaxy_names, folders_out = get_galaxy_names_list(in_dir)

In [ ]:
output_df = pd.DataFrame()

for filepath, gname in zip(folders_out, galaxy_names):
    gal_dict, param_names = galfit_param_grab(pj(filepath, gname + ".in"))
    if not param_names: continue
    
    gname_df = flatten_to_pandas(gal_dict, param_names, gname)
    output_df = pd.concat([output_df, gname_df])
    
#galfit_in_df = output_df
#galfit_out_df = flatten_to_pandas(galfit_param_grab(), param_names, gname)

In [ ]:
galfit_in_df = output_df

In [ ]:
# output_df = pd.DataFrame()

# for filepath, gname in zip(folders_out, galaxy_names):
#     gal_dict, param_names = galfit_param_grab(pj(filepath, gname + ".out"))
#     if not param_names: continue
    
#     gname_df = flatten_to_pandas(gal_dict, param_names, gname)
#     output_df = pd.concat([output_df, gname_df])

# galfit_out_df = output_df
# #galfit_out_df = flatten_to_pandas(galfit_param_grab(), param_names, gname)

In [ ]:
ignore_galfit_in, ignore_galfit_out, in_filter = export_filter()

In [ ]:
ignore_galfit_out

In [ ]:
#galfit_in_df.drop(columns = ignore_galfit_in)
# For the input galaxies, we have a lot of held values, these are uneccessary
# https://stackoverflow.com/a/39658662
nunique = galfit_in_df.nunique()
cols_to_drop = list(nunique[nunique == 1].index) + ignore_galfit_in
reduced_galfit_in_df = galfit_in_df.drop(columns = cols_to_drop)

In [ ]:
galfit_in_df = output_df
galfit_in_df = galfit_in_df.drop(columns = "Crop Rad")

In [ ]:
# Do this to grab just the columns we want
leftover = list(set(galfit_in_df.columns).difference(set(ignore_galfit_in + ignore_galfit_out)))
# Do this to retain column order
leftover = [col for col in galfit_in_df.columns if col in leftover]
# Manual removal for now (since galfit changed names on me)
try:
    leftover.remove('Spiral outer (i.e. asymptotic) radius (pixels) disk')
except ValueError:
    pass

In [ ]:
pred_out = model_xgb.predict(reduced_galfit_in_df)
pred_out_df = pd.DataFrame(pred_out, columns = leftover, index = reduced_galfit_in_df.index)

In [ ]:
galfit_in_df.update(pred_out_df)

In [ ]:
too_small = galfit_in_df.query("`Asymptotic spiral powerlaw disk` < -5")
too_small.loc[:,"Asymptotic spiral powerlaw disk"] = 1
galfit_in_df.update(too_small)
# use this as a prefilter for training and then we won't have to do this here ;)

In [ ]:
leftover

In [ ]:
for col in galfit_in_df.columns:
    if col in leftover and ("degree" in col.lower() or "angle" in col.lower()):
        galfit_in_df[col] *= 180/np.pi

In [ ]:
galfit_in_df["Inclination to L.o.S. (degrees) disk"] *= 180/np.pi

In [ ]:
# The interim step until I feel confident enough in the model to step in *before* outputting these all to file
def update_feedmes(new_in_df, top_dir = ""):
    
    in_dir, tmp_dir, out_dir = command_line(top_dir)
    filenames_fits_in, galaxy_names, folders_out = get_galaxy_names_list(in_dir)
    
    psf_info = csv_sdss_info(galaxy_names)
    
    count = 0
    paths_to_feedme = []
    
    for galaxy in folders_out:
    
        gname = galaxy_names[count]
        print(gname)
        
        galaxy_info = new_in_df.loc[gname, :]
        
        if(os.path.basename(galaxy) != gname):
            print("uh oh naming went wrong")
            sys.exit()
            
        bulge_rad, bulge_axis_ratio, pos_angle_bulge, \
            crop_rad, center_pos_x, center_pos_y, \
            disk_maj_axs_len, pos_angle_disk, pos_angle_power, \
            axis_ratio, max_arc, spin_dir, \
            est_arcs, inclination, bar_candidate, \
            alpha = galaxy_information(gname, galaxy)
        
        center_pos_x = float(center_pos_x)
        center_pos_y = float(center_pos_y)
        crop_rad = float(crop_rad)
        
        x1crop = round(center_pos_x - crop_rad)
        x2crop = round(center_pos_x + crop_rad)        
        y1crop = round(center_pos_y - crop_rad)
        y2crop = round(center_pos_y + crop_rad)
               
        # Initializing Feedme
        feedme_list = []
        
        # Initialize template dict
        gt = quick_build_template() # galfit template
        #gt = rebuild_template_dict("./m51.feedme")
    
        #To reconstruct the z PSF (i.e., the 5th HDU) at the position (row, col) = (500, 600) from run 1336, column 2, field 51 you’d say:
        #read_PSF psField-001336-2-0051.fit 5 500.0 600.0 foo.fit
        run, rerun, camcol, field, psf_row, psf_col, petromag = psf_info[gname]
        
        # new_in_df shouldddd retain order of columns, making my life (the programmer's) easier
        # hardcoding those should provide a good litmus test too
        all_col = new_in_df.columns# .drop("Crop Rad").values

        feedme_list.append(f"#{run}{camcol}{field}; HDU: z{psf_row}{psf_col}")
        feedme_list.append("")
        # Image and Galfit Control Param
        feedme_list.append(f"A) {filenames_fits_in[count]}")
        feedme_list.append(f"B) {tmp_dir}/galfits/{gname}_out.fits")
        feedme_list.append(f"C) none")
        # Commenting out psf for now because it's causing galfit to crash
        feedme_list.append(f"D) none") #{tmp_dir}/psf_files/{gname}_psf.fits")
        feedme_list.append(f"E) 1")
        feedme_list.append(f"F) {tmp_dir}/galfit_masks/{gname}_star-rm.fits")
        feedme_list.append(f"G) none")  #./constraints.txt"
        feedme_list.append(f"H) {x1crop:d} {x2crop:d} {y1crop:d} {y2crop:d}")
        feedme_list.append(f"I) 50 50") # psf FWHM ~= 1, Chien recommends 40-80 times this value
        feedme_list.append(f"J) 24.8") # SDSS
        feedme_list.append(f"K) 0.396  0.396") # SDSS
        feedme_list.append(f"O) regular")
        feedme_list.append(f"P) 0")
        feedme_list.append("")
        
        # Sersic 1
        # Fixing as much as I can here... it's not exactly a priority.
        feedme_list.append(f"# Component number: 1")
        feedme_list.append(f"0) sersic")
        feedme_list.append(f"1) {center_pos_x:.1f} {center_pos_y:.1f} 0 0")
        feedme_list.append(f"3) {galaxy_info[all_col[0]]:.2f} 1") # Initial guess goes here
        feedme_list.append(f"4) {galaxy_info[all_col[1]]:.2f} 1") 
        feedme_list.append(f"5) {galaxy_info[all_col[2]]:.2f} 1") # According to other paper GALFIT usually doesn't have a problem with the index
        feedme_list.append("6) 0  0")    
        feedme_list.append("7) 0  0")    
        feedme_list.append("8) 0  0")    
        # According to other papers, bulge (esp. in spiral galaxies) averages to about 2 so this is a good starting place
        # see https://ned.ipac.caltech.edu/level5/Sept11/Buta/Buta9.html
        feedme_list.append(f"9) {galaxy_info[all_col[3]]:.2f} 1")  
        feedme_list.append(f"10) {galaxy_info[all_col[4]]:.2f} 1") 
        feedme_list.append("")
    
        # Sersic 2
        feedme_list.append("# Component number: 2")
        feedme_list.append(f"0) sersic")
        feedme_list.append(f"1) {center_pos_x:.1f} {center_pos_y:.1f} 0 0")
        feedme_list.append(f"3) {galaxy_info[all_col[5]]:.2f} 1") 
        feedme_list.append(f"4) {galaxy_info[all_col[6]]:.2f} 1") # Use this for effective radius? Also 0 this one out? Will have to see how well it works in practice
        feedme_list.append(f"5) {galaxy_info[all_col[7]]:.2f} 1") # Classical disk follows Sersic n = 1 so good place to start (per Readme Exponential profile)
                                      # According to comparison tests, this usually ends up much higher probably due to the spiral.
        feedme_list.append("6) 0  0")    
        feedme_list.append("7) 0  0")    
        feedme_list.append("8) 0  0")    
        feedme_list.append(f"9) 0.6 0") #{galaxy_info[all_col[8]]:.2f} 1")  # Fixing this to 0.6 to give the arms the best chance to form
        #(f"9) {axis_ratio - 0.3} 1 {gt['9']}")
        feedme_list.append(f"10) {galaxy_info[all_col[9]]:.2f} 1") #90  1") # fixing this to 'normal' 0 so that we can JUST rotate power function
        #feedme_list.append(f"10) 90  1") # fixing this to 'normal' 0 so that we can JUST rotate power function
        feedme_list.append("")
    
        # Power
        feedme_list.append("R0) power")
        feedme_list.append(f"R1) {galaxy_info[all_col[10]]:.2f} 0") # Chosen based on where *detection* of arms usually start
        feedme_list.append(f"R2) {galaxy_info[all_col[11]]:.2f} 0")
        feedme_list.append(f"R3) {galaxy_info[all_col[12]]:.2f} 1") # See calc above
        feedme_list.append(f"R4) {galaxy_info[all_col[13]]:.2f} 1") # Another good thing to automate via Sparcfire 
        feedme_list.append(f"R9) {galaxy_info[all_col[14]]:.2f} 1") # see if can't 0 this one out... 
        feedme_list.append(f"R10) {galaxy_info[all_col[15]]:.2f}  1")# 40 + pos_angle_power + " 1") # Always more to discover, looks like all the images are mirrored across the y axis.

        # ---- Fourier modes. May need to add more at some point (?)
        feedme_list.append(f"F1) {galaxy_info[all_col[16]]:.2f} 45  1  1") # Need to experiment with amplitude and phase angle for better understanding of this
        feedme_list.append(f"F3) {galaxy_info[all_col[17]]:.3f} 25  1  1")
        feedme_list.append(f"#F4) {galaxy_info[all_col[18]]:.3f} 4  1  1")
        feedme_list.append(f"#F5) {galaxy_info[all_col[19]]:.3f} 6  1  1")  
        feedme_list.append("")
    
        # Sky -- Necessary?
        feedme_list.append(f"# Component number: 3")
        feedme_list.append(f"0) sky")
        feedme_list.append(f"1) 1000  1")
        feedme_list.append(f"2) 0  1")
        feedme_list.append(f"3) 0  1")
                       
        count += 1
        
        formatted_feedme = []
        extra = ""
        for i in feedme_list:
            if i and not i.startswith("#"):
                str_split = i.split(")")
                component = extra + str_split[0]
                formatted_feedme.append(f"{i:<{gt['fill']}} {gt[component]}")
                
                # Sneakily do this at the end since 0) sky is just component name
                if "sky" in str_split[1]:
                    extra = "sky"
        #_ = [print(i) for i in formatted_feedme]
        #paths_to_feedme.append(write_to_feedme(galaxy, formatted_feedme, feedme_name = gname + "_input")) # do I need paths_to_feedme? I used to use it for something...
        paths_to_feedme.append(write_to_feedme(pj(top_dir, out_dir, gname), formatted_feedme, feedme_name = gname + ".in")) # do I need paths_to_feedme? I used to use it for something...

In [ ]:
update_feedmes(galfit_in_df, top_dir = top_dir)